In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

service = Service(ChromeDriverManager().install())
options = Options()
# options.add_argument('--headless')
# options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--window-size=1920x1080')
options.add_argument('--start-maximized')
options.add_argument('--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36')
options.add_argument('--lang=ko_KR')

driver = webdriver.Chrome(service=service, options=options)

driver.get("https://www.musinsa.com/main/musinsa/recommend?gf=F")
time.sleep(3)

search_button = driver.find_element(By.CLASS_NAME, "sc-dzagxs-2")
search_button.click()
time.sleep(2)

popular_keywords = driver.find_elements(By.CSS_SELECTOR, ".search-home-popular-wrap .sc-1xzm8bl-1")
target_keyword = popular_keywords[2].text
print("목표키워드 :", target_keyword)

search_input = driver.find_element(By.CLASS_NAME, "search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword)
time.sleep(1)

search_input.send_keys(Keys.RETURN) # 엔터를 자동으로 치기 위해 
time.sleep(5)

print("검색 완료. 현재 페이지 타이틀 :", driver.title)
driver.quit()

목표키워드 : 반팔
검색 완료. 현재 페이지 타이틀 : 반팔 | 무신사 추천 상품


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re # remove의 약자 -> 무엇인가를 제거하겠다.


service = Service(ChromeDriverManager().install())
options = Options()
# options.add_argument('--headless')
# options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--window-size=1920x1080')
options.add_argument('--start-maximized')
options.add_argument('--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36')
options.add_argument('--lang=ko_KR')

driver = webdriver.Chrome(service=service, options=options)

driver.get("https://www.musinsa.com/main/musinsa/recommend?gf=F")
time.sleep(3)

search_button = driver.find_element(By.CLASS_NAME, "sc-dzagxs-2")
search_button.click()
time.sleep(2)

popular_keywords = driver.find_elements(By.CSS_SELECTOR, ".search-home-popular-wrap .sc-1xzm8bl-1")
target_keyword = popular_keywords[2].text
print("목표키워드 :", target_keyword)

search_input = driver.find_element(By.CLASS_NAME, "search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword)
time.sleep(1)

search_input.send_keys(Keys.RETURN) # 엔터를 자동으로 치기 위해 
time.sleep(5)

print("검색 완료. 현재 페이지 타이틀 :", driver.title)

scroll_pause = 2

for _ in range(10) :
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause)

products = driver.find_elements(By.CSS_SELECTOR, "div.sc-d36st-1.elXjMR")[:100]
product_data = []

for product in products :
    try :
        name = product.find_element(By.CSS_SELECTOR, "div.sc-cMuefe.jIlLeV a.gtm-select-item span").text.strip()
        url = product.find_element(By.CSS_SELECTOR, "div.sc-cMuefe.jIlLeV a.gtm-select-item").get_attribute("href")
        review_info = product.find_elements(By.CSS_SELECTOR, "span.text-etc_11px_reg.text-yellow.font-pretendard")[1].text
        review_count =  int(re.sub(r"[^\d]", "", review_info)) if review_info else 0 # \d : 숫자를 의미. -> ^ : 반대의 의미. 숫자가 아닌 것을 찾겠다. row 스트링의 약자 : r
        product_data.append({"상품명": name, "URL": url, "리뷰수": review_count})
    except :
        continue
        
df = pd.DataFrame(product_data)
df_top30 = df.sort_values(by="리뷰수", ascending=False).head(30).reset_index(drop=True) # 리뷰가 많은 순으로 30개를 쪼갰는데 나머지는 버려라.

driver.quit()
df_top30.head()

목표키워드 : 반팔
검색 완료. 현재 페이지 타이틀 : 반팔 | 무신사 추천 상품


,상품명,URL,리뷰수
0,선샤인 오버핏 반팔 티셔츠 GT-360,https://www.musinsa.com/products/4116793,954
1,Retro Motorcycle Graphic Tee White,https://www.musinsa.com/products/4241149,926
2,Vintage Rose Graphic T-shirt VW4SE027_3color,https://www.musinsa.com/products/3956456,442
3,오버핏 크루넥 1992 반팔 티셔츠 화이트블랙,https://www.musinsa.com/products/4082110,426
4,TIGER HALF SLEEVE TEE-NAVY,https://www.musinsa.com/products/3062741,358


In [7]:
# 리뷰를 찾아서 엑셀에 넣는 방법
from openpyxl import Workbook

driver = webdriver.Chrome(service=service, options=options)
all_reviews = []

for idx, row in df_top30.iterrows() :
    print(f"{idx + 1}번째 상품 리뷰 수집 중 :", {row['상품명']})
    driver.get(row['URL'])
    time.sleep(2)

    try :
        review_tab = driver.find_element(By.XPATH, "//button[@data-button-id='prd_review_tab']")
        review_tab.click()
        time.sleep(2)

        for _ in range(3) :
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)

        review_elements = driver.find_elements(By.CSS_SELECTOR, ".text-body_13px_reg.text-black.font-pretendard")
        for review in review_elements[:10] :
            text = review.text.strip()
            if text :
                all_reviews.append({
                    "상품명": row["상품명"],
                    "리뷰": text
                })
    except Exception as e :
        print("리뷰 수집 실패:", e)
        continue

driver.quit()

wb = Workbook()
ws = wb.active
ws.title = "무신사 리뷰"

ws.append(["상품명", "리뷰"])

for r in all_reviews :
    ws.append([r["상품명"], r["리뷰"]])

wb.save("musinsa_top30_reviews.xlsx")
print("✅ 엑셀 저장 완료 : musinsa_top30_reviews.xlsx")

1번째 상품 리뷰 수집 중 : {'선샤인 오버핏 반팔 티셔츠 GT-360'}
리뷰 수집 실패: Message: element click intercepted: Element <button type="button" class="sc-g3hx4t-2 eUVhgj gtm-click-button" data-index="(not set)" data-section-name="prd_tabs" data-section-index="12" data-brand-id="(not set)" data-button-id="prd_review_tab" data-button-name="스냅·후기탭클릭">...</button> is not clickable at point (826, 823). Other element would receive the click: <span class="text-xs font-normal text-blue font-pretendard" data-mds="Typography">...</span>
  (Session info: chrome=138.0.7204.184); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementclickinterceptedexception
Stacktrace:
0   chromedriver                        0x00000001025f0b38 cxxbridge1$str$ptr + 2722088
1   chromedriver                        0x00000001025e8aa8 cxxbridge1$str$ptr + 2689176
2   chromedriver                        0x000000010213a33c cxxbridge1$string$len + 90648
3   chromedriver      

In [8]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine

EXCEL_PATH = "musinsa_top30_reviews.xlsx"
DB_NAME = "musinsa_db_v3"
TABLE_NAME = "reviews"
USER = "root"
PASSWORD = "whgPfls3770"
HOST = "127.0.0.1"
PORT = 3306

df = pd.read_excel(EXCEL_PATH)

conn = pymysql.connect(host=HOST, user=USER, password=PASSWORD, port=PORT)
cursor = conn.cursor()
cursor.execute(
    f"CREATE DATABASE IF NOT EXISTS {DB_NAME} CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;"
)
conn.close()

engine_str = f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}?charset=utf8mb4"
engine = create_engine(engine_str)

df.to_sql(name=TABLE_NAME, con=engine, if_exists="replace", index=False)

print(f"✅ 데이터가 MySQL [{DB_NAME}]의 테이블 [{TABLE_NAME}]에 성공적으로 저장되었습니다.")

✅ 데이터가 MySQL [musinsa_db_v3]의 테이블 [reviews]에 성공적으로 저장되었습니다.


In [9]:
pip install gspread

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [gspread]
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install oauth2client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [oauth2client]
Note: you may need to restart the kernel to use updated packages.


In [13]:
from konlpy.tag import Okt
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd

df = pd.read_excel("musinsa_top30_reviews.xlsx")

okt = Okt()
df["리뷰_키워드"] = df["리뷰"].apply(lambda x: ", ".join(okt.nouns(str(x))))# lambda : 즉각적인 함수를 만들어 낼 수 있는?

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"] # 범위
creds = ServiceAccountCredentials.from_json_keyfile_name("lunar-goal-468703-f1-6833eb2d1b4f.json", scope)
client = gspread.authorize(creds)

spreadsheet = client.open("무신사 리뷰 분석_v1")
sheet1 = spreadsheet.sheet1  # 현재 열려있는 시트 1
sheet1.clear() # 초기화
sheet1.update([df.columns.values.tolist()] + df.values.tolist()) # 값을 추가한다. 

try :
    spreadsheet.add_worksheet(title="키워드 분석", rows="1000", cols="20")
except :
    pass

sheet2 = spreadsheet.worksheet("키워드 분석")
sheet2.clear()
sheet2.update([["상품명", "리뷰", "리뷰_키워드"]] + df[["상품명", "리뷰", "리뷰_키워드"]].values.tolist())

print("✅ 구글 스프레드시트에 원본 + 키워드 분석자료 업데이트 완료!")

✅ 구글 스프레드시트에 원본 + 키워드 분석자료 업데이트 완료!
